# Importando bibliotecas

In [220]:
import pandas as pd
import numpy as np
import os, re
from pathlib import Path

In [221]:
pd.set_option('display.max_columns', 90)
pd.set_option('display.max_rows', 100)

# Variáveis

In [222]:
dataset_folder = r'C:\Users\GabGalani\Documents\Data Projects\Analise_credito\dataset\bronze'
file_test = 'test.csv'

# Datasets

## Dataset test.csv

In [223]:
dataset = pd.read_csv(os.path.join(dataset_folder, file_test), sep=',')
dataset.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,4,12.27,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",4,5,11.27,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,1,5.42,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [224]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      44985 non-null  object 
 4   Age                       50000 non-null  object 
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  object 
 8   Monthly_Inhand_Salary     42502 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  int64  
 12  Num_of_Loan               50000 non-null  object 
 13  Type_of_Loan              44296 non-null  object 
 14  Delay_

## Verificando e tratando nulos

In [225]:
dataset.isnull().sum()

ID                             0
Customer_ID                    0
Month                          0
Name                        5015
Age                            0
SSN                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary       7498
Num_Bank_Accounts              0
Num_Credit_Card                0
Interest_Rate                  0
Num_of_Loan                    0
Type_of_Loan                5704
Delay_from_due_date            0
Num_of_Delayed_Payment      3498
Changed_Credit_Limit           0
Num_Credit_Inquiries        1035
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Credit_History_Age          4470
Payment_of_Min_Amount          0
Total_EMI_per_month            0
Amount_invested_monthly     2271
Payment_Behaviour              0
Monthly_Balance              562
dtype: int64

- Coluna Name nula, deverá ser preenchida, pois há casos como o acima, onde o ID já possui o nome; (Essa é irrelevante)
- Monthly_Inhand_Salary é importante para avaliar a capacidade de pagamento, então trataei os nulos com a mediana da coluna       Annual_Income;
- Num_of_Delayed_Payment é importante pois avalia o comportamento do pagamento, então assumiremos como 0;
- Num_Credit_Inquiries é importante pois mostra quantas consultas de crédito foram feitas recentemente, vamos considerar como 0;
- Credit_History_Age é importante pois indica o histórico de crédito, vamos considerar fazer um dataset para verificar se já possui essa informação em outra linha e agregar calculando o tempo
- Amount_invested_monthly Indica quanto o cliente investe mensalmente, vamos conseidarar 0 para os nulos devido à cada linha ser um mês o nulo indica que não houve depósito

## Tratando a coluna Age

In [226]:
dataset.head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196


In [227]:
dataset['Age'] = dataset['Age'].str.replace('_', '', regex=False).astype(float)
dataset['Age'] = dataset['Age'].astype(int)


## Coluna Name

In [228]:
dataset_name = dataset[['Customer_ID', 'Name']]
dataset_name = dataset_name[(~dataset_name['Name'].isnull()) & (~dataset_name['Customer_ID'].isnull())]
dataset_name = dataset_name.drop_duplicates()

dataset = dataset.merge(dataset_name, on='Customer_ID', how='left', suffixes=('', '_new'))
dataset['Name'] = dataset['Name'].fillna(dataset['Name_new'])
dataset['Name'] = dataset['Name'].fillna(dataset['Name_new'])

## Coluna Credit_History_Age

In [229]:
def convert_to_months(age_str):
    if pd.isnull(age_str):
        return None
    parts = age_str.split(' ')
    years = int(parts[0])
    months = int(parts[3])
    return years * 12 + months

def convert_to_years_months(months):
    if pd.isnull(months):
        return "0 Years and 0 Months"  # Retornar um valor padrão para NaN
    years = months // 12
    rem_months = months % 12
    return f"{int(years)} Years and {int(rem_months)} Months"

In [230]:

dataset_credit_history = dataset[['Customer_ID', 'Month', 'Age', 'Credit_History_Age']]

dataset_credit_history['Month_number'] = pd.to_datetime(dataset_credit_history['Month'], format='%B')
dataset_credit_history = dataset_credit_history.sort_values(by=['Customer_ID', 'Month'])

# Converte o 'Credit_History_Age' para 'Credit_History_Months'
dataset_credit_history['Credit_History_Months'] = dataset_credit_history['Credit_History_Age'].apply(convert_to_months)

# Preencher valores nulos
dataset_credit_history['Credit_History_Months'] = dataset_credit_history['Credit_History_Months'].fillna(dataset_credit_history['Credit_History_Months'].shift(1) - 1)

# Preencher valores nulos restantes usando bfill
dataset_credit_history['Credit_History_Months'] = dataset_credit_history.groupby('Customer_ID')['Credit_History_Months'].bfill()

# Convertendo novamente para o formato 'Years and Months'
dataset_credit_history['Credit_History_Age'] = dataset_credit_history['Credit_History_Months'].apply(convert_to_years_months)

# Exibindo os resultados
dataset_credit_history.head(2)


C:\Users\GabGalani\AppData\Local\Temp\ipykernel_11668\3128593773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_credit_history['Month_number'] = pd.to_datetime(dataset_credit_history['Month'], format='%B')


,Customer_ID,Month,Age,Credit_History_Age,Month_number,Credit_History_Months
28379,CUS_0x1000,December,18,11 Years and 1 Months,1900-12-01,133.0
28378,CUS_0x1000,November,18,11 Years and 0 Months,1900-11-01,132.0


Ajustando a coluna tratada

In [231]:
dataset = dataset.merge(dataset_credit_history, on=['Customer_ID', 'Month', 'Age'], how='left', suffixes=('', '_new'))
dataset['Credit_History_Age'] = dataset['Credit_History_Age'].fillna(dataset['Credit_History_Age_new'])
dataset['Credit_History_Age'] = dataset['Credit_History_Age'].fillna(dataset['Credit_History_Age_new'])

In [232]:
def extract_year(age_str):
    # Verificar se age_str é uma string antes de aplicar split
    if isinstance(age_str, str):
        parts = age_str.split(' ')
        years = int(parts[0])
        return years+1
    return 0  # Retornar None caso não seja uma string


dataset['Age_new'] = dataset['Credit_History_Age'].apply(extract_year).astype(int)

dataset['Age'] = np.where(
    (dataset['Age'] >= 100) | (dataset['Age'] < 0),
    dataset['Age_new'],
    dataset['Age']
)

## Monthly_Inhand_Salary 

Importante para avaliar capacidade de pagamento. <br>
Preencher com a mediana ou calcular com base na Annual_Income

In [233]:
dataset['Annual_Income'] = dataset['Annual_Income'].str.replace('_', '', regex=False).astype(float)
dataset['Annual_Income'] = dataset['Annual_Income'].astype(int)
dataset['Annual_Income'].astype(float)
dataset['Monthly_Inhand_Salary'] = dataset['Monthly_Inhand_Salary'].fillna(dataset['Annual_Income'] / 12)

## Num_of_Delayed_Payment

Avalia comportamento de pagamento. <br>
Preencher com 0 (assumir que não houve atraso) ou com a mediana

In [234]:
dataset['Num_of_Delayed_Payment'] = dataset['Num_of_Delayed_Payment'].fillna(0)

## Num_Credit_Inquiries

Mostra quantas consultas ao crédito foram feitas (uso frequente pode ser sinal de risco). <br>
Preencher com a mediana ou 0:

In [235]:
dataset['Num_Credit_Inquiries'] = dataset['Num_Credit_Inquiries'].fillna(0)


## Amount_invested_monthly

Indica quanto o cliente investe por mês, relevante para estabilidade financeira. <br>
Preencher com 0 para o mês nulo


In [236]:
dataset['Amount_invested_monthly'] = dataset['Amount_invested_monthly'].fillna(0)

## Monthly_Balance

Saldo mensal restante após despesas e pagamentos. <br>
Preencher com mediana

In [237]:
dataset['Monthly_Balance'] = pd.to_numeric(dataset['Monthly_Balance'], errors='coerce')
dataset['Monthly_Balance'] = dataset['Monthly_Balance'].fillna(dataset['Monthly_Balance'].mean())

## Type_of_Loan 

Pode indicar o tipo de empréstimo obtido (ex.: pessoal, carro, etc.). <br>
|Preencher com "Unknown"

In [238]:
dataset['Type_of_Loan'] = dataset['Type_of_Loan'].fillna('')

## Colunas numericas e float

In [239]:
int_columns = ['Age', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries']
float_columns = ['Annual_Income', 'Monthly_Inhand_Salary', 'Interest_Rate', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

Transformando todo dataset em string

In [240]:
for i in dataset.columns: 
    dataset[i] = dataset[i].astype(str)

Colunas int

In [241]:
for i in int_columns:
    dataset[i] = dataset[i].str.replace('_', '', regex=False).astype(float)
    dataset[i] = dataset[i].astype(int)

Colunas float

In [242]:
dataset['Changed_Credit_Limit'] = dataset['Changed_Credit_Limit'].replace('_', '0')
float_columns = ['Annual_Income', 
                 'Monthly_Inhand_Salary',
                 'Interest_Rate', 
                 'Changed_Credit_Limit', 
                 'Outstanding_Debt', 
                 'Credit_Utilization_Ratio', 
                 'Total_EMI_per_month', 
                 'Amount_invested_monthly', 
                 'Monthly_Balance'
            ]

In [243]:
for i in float_columns:
    dataset[i] = dataset[i].str.replace('_', '', regex=False).astype(float)
    dataset[i] = dataset[i].astype(float)

## Verificando as colunas tratadas

In [244]:
dataset.isnull().sum()

ID                          0
Customer_ID                 0
Month                       0
Name                        0
Age                         0
SSN                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Name_new                    0
Credit_History_Age_new      0
Month_number                0
Credit_History_Months       0
Age_new                     0
dtype: int64

## Verificando valor _______ na coluna Occupation

In [245]:
dataset['Occupation'].unique()

array(['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur',
       'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect'], dtype=object)

In [246]:
dataset[dataset['Occupation'] == '_______'].head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Name_new,Credit_History_Age_new,Month_number,Credit_History_Months,Age_new
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,1,5.42,5,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434,Rick Rothackerj,27 Years and 3 Months,1900-09-01,327.0,28
41,0x1683,CUS_0xa66b,October,Saphirj,41,221-30-8554,_______,33751.0,2812.583333,5,5,20.0,3,"Credit-Builder Loan, Personal Loan, and Auto Loan",16,20,11.00,0,Standard,1328.93,36.778209,19 Years and 11 Months,Yes,65.008174,139.303039,Low_spent_Small_value_payments,380.549371,Saphirj,19 Years and 11 Months,1900-10-01,239.0,20


In [247]:
dataset[dataset['Customer_ID'] == 'CUS_0x21b1'].head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Name_new,Credit_History_Age_new,Month_number,Credit_History_Months,Age_new
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,1,5.42,5,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434,Rick Rothackerj,27 Years and 3 Months,1900-09-01,327.0,28
5,0x1617,CUS_0x21b1,October,Rick Rothackerj,28,#F%$D@*&8,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,3,5.42,5,Good,605.03,30.116600,27 Years and 4 Months,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083,Rick Rothackerj,27 Years and 4 Months,1900-10-01,328.0,28


In [248]:
dataset_occupation = dataset[['Customer_ID', 'Occupation']]
dataset_occupation = dataset_occupation[dataset_occupation['Occupation'] != "_______"]
dataset_occupation = dataset_occupation.drop_duplicates()

dataset = dataset.merge(dataset_occupation, on='Customer_ID', how="left", suffixes=('', '_new'))
dataset['Occupation'] = np.where(
    dataset['Occupation'] == '_______',
    dataset['Occupation_new'],
    dataset['Occupation']
)

In [249]:
dataset['Occupation'].unique()

array(['Scientist', 'Teacher', 'Engineer', 'Entrepreneur', 'Developer',
       'Lawyer', 'Media_Manager', 'Doctor', 'Journalist', 'Manager',
       'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect'],
      dtype=object)

## Tratando valor !@9#%8 na coluna Payment_Behaviour

In [250]:
dataset['Payment_Behaviour'].unique()

array(['Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments',
       'Low_spent_Medium_value_payments',
       'High_spent_Large_value_payments',
       'Low_spent_Large_value_payments', '!@9#%8',
       'High_spent_Small_value_payments'], dtype=object)

In [251]:
dataset[dataset['Payment_Behaviour'] == '!@9#%8'].head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Name_new,Credit_History_Age_new,Month_number,Credit_History_Months,Age_new,Occupation_new
7,0x1619,CUS_0x21b1,December,Rick Rothackerj,28,004-07-5839,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,2,7.42,5,_,605.03,33.875167,27 Years and 6 Months,No,18.816215,153.534488,!@9#%8,421.447964,Rick Rothackerj,27 Years and 6 Months,1900-12-01,330.0,28,Teacher
36,0x1676,CUS_0xba08,September,Jamesj,45,366-68-1681,Journalist,31370.0,2825.233333,1,6,12.0,2,"Not Specified, and Payday Loan",3,0,-0.24,5,Good,421.43,40.633131,27 Years and 1 Months,No,46.616129,161.583377,!@9#%8,364.323828,Jamesj,27 Years and 1 Months,1900-09-01,325.0,28,Journalist


In [252]:
dataset[dataset['Customer_ID'] == 'CUS_0x21b1'].head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Name_new,Credit_History_Age_new,Month_number,Credit_History_Months,Age_new,Occupation_new
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,1,5.42,5,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434,Rick Rothackerj,27 Years and 3 Months,1900-09-01,327.0,28,Teacher
5,0x1617,CUS_0x21b1,October,Rick Rothackerj,28,#F%$D@*&8,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,3,5.42,5,Good,605.03,30.116600,27 Years and 4 Months,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083,Rick Rothackerj,27 Years and 4 Months,1900-10-01,328.0,28,Teacher
6,0x1618,CUS_0x21b1,November,Rick Rothackerj,28,004-07-5839,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,0,5.42,5,_,605.03,30.996424,27 Years and 5 Months,No,18.816215,72.680145,High_spent_Large_value_payments,452.302307,Rick Rothackerj,27 Years and 5 Months,1900-11-01,329.0,28,Teacher
7,0x1619,CUS_0x21b1,December,Rick Rothackerj,28,004-07-5839,Teacher,34847.0,3037.986667,2,4,6.0,1,Credit-Builder Loan,3,2,7.42,5,_,605.03,33.875167,27 Years and 6 Months,No,18.816215,153.534488,!@9#%8,421.447964,Rick Rothackerj,27 Years and 6 Months,1900-12-01,330.0,28,Teacher


In [253]:
def classify_payment_behaviour(row):
    total_spent = row['Total_EMI_per_month'] + row['Amount_invested_monthly'] + row['Outstanding_Debt']
    salary = row['Monthly_Inhand_Salary']
    
    if total_spent > 0.6 * salary:
        spending = 'High_spent'
    else:
        spending = 'Low_spent'
    
    if total_spent < 500:
        payment_value = 'Small_value_payments'
    elif total_spent < 2000:
        payment_value = 'Medium_value_payments'
    else:
        payment_value = 'Large_value_payments'

    return f'{spending}_{payment_value}'

In [254]:
dataset_teste = dataset
dataset_teste['Payment_Behaviour_new'] = dataset_teste.apply(classify_payment_behaviour, axis=1)
dataset['Payment_Behaviour'] = np.where(
    dataset['Payment_Behaviour'] == '!@9#%8',
    dataset['Payment_Behaviour_new'],
    dataset['Payment_Behaviour']
)

dataset['Payment_Behaviour'].unique()

array(['Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments',
       'Low_spent_Medium_value_payments',
       'High_spent_Large_value_payments',
       'Low_spent_Large_value_payments',
       'High_spent_Small_value_payments'], dtype=object)

## Tratando valores _ na coluna Credit_Mix

In [255]:
dataset['Credit_Mix'].unique()

array(['Good', '_', 'Standard', 'Bad'], dtype=object)

In [256]:
dataset = dataset.sort_values(by=['Customer_ID', 'Month', 'Age'])
dataset['Credit_Mix_new'] = dataset['Credit_Mix'].replace('_', None).fillna(method='bfill')

dataset['Credit_Mix'] = np.where(
    dataset['Credit_Mix'] == '_',
    dataset['Credit_Mix_new'],
    dataset['Credit_Mix']
)
dataset['Credit_Mix'].unique()

C:\Users\GabGalani\AppData\Local\Temp\ipykernel_11668\282542408.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset['Credit_Mix_new'] = dataset['Credit_Mix'].replace('_', None).fillna(method='bfill')


array(['Bad', 'Standard', 'Good'], dtype=object)

## Tratando valores errados coluna Num_of_Loan

In [257]:
dataset['Num_of_Loan_new'] = dataset['Type_of_Loan'].apply(lambda x: len(x.split(',')))
dataset['Num_of_Loan'] = np.where(
    dataset['Type_of_Loan'] == 'Not Specified',
    1,
    dataset['Num_of_Loan']
)

dataset['Num_of_Loan'] = np.where(
    dataset['Type_of_Loan'] != dataset['Num_of_Loan_new'],
    dataset['Num_of_Loan_new'],
    dataset['Num_of_Loan']
)

dataset['Num_of_Loan'] = np.where(
    dataset['Type_of_Loan'] == '',
    0,
    dataset['Num_of_Loan']
)

## Excluindo as colunas _new

In [258]:
for i in dataset.columns:
    if "_new" in i or "Month_number" == i or 'Credit_History_Months' == i:
        dataset.drop(columns=i, inplace=True)

In [259]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 28379 to 49576
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      50000 non-null  object 
 4   Age                       50000 non-null  int64  
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  float64
 8   Monthly_Inhand_Salary     50000 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  float64
 12  Num_of_Loan               50000 non-null  int64  
 13  Type_of_Loan              50000 non-null  object 
 14  Delay_f

# Criando a pasta silver e salvando o novo dataset

In [260]:
silver_folder = r'C:\Users\GabGalani\Documents\Data Projects\Analise_credito\dataset\silver'
Path(silver_folder).mkdir(parents=True, exist_ok=True)

In [261]:
dataset.to_csv(os.path.join(silver_folder, 'silver_test.csv'), sep=';', index=False)